In [1]:
import pandas as pd
import os

def generate_bash_script(mode, aqm, script_dir="scripts", config_dir="configs"):
    """
    Reads data from a CSV file and generates a bash script to apply tc rules iteratively.
    Mode can be 'Baseline' or 'Best'.
    AQM can be fq_codel, codel, cake, fq_pie, or pfifo.
    Saves scripts in script_dir and CSV configs in config_dir.
    """

    aqm_map = {
        "fq_codel": "fq_codel",
        "codel": "codel",
        "cake": "cake",
        "fq_pie": "fq_pie",
        "pfifo": "pfifo",
        "dualpi2": "dualpi2"
    }

    if aqm not in aqm_map:
        print(f"Error: Unsupported AQM '{aqm}'")
        return

    if mode not in ["Baseline", "Best"]:
        print("Error: mode must be either 'Baseline' or 'Best'")
        return

    # Ensure output directories exist
    os.makedirs(script_dir, exist_ok=True)
    os.makedirs(config_dir, exist_ok=True)

    csv_file_path = f"{mode}_Satellite_Australia_Simulation_Log_cleaned_starlink_downlink.csv"
    script_filename = os.path.join(script_dir, f"apply_tc_rules_{mode.lower()}_{aqm}.sh")
    config_filename = os.path.join(config_dir, f"router_configs_{mode.lower()}_{aqm}.csv")

    required_columns = [
        f'Sydney_{mode}_Thrpt',
        f'Sydney_{mode}_Latency',
        f'Sydney_{mode}_BER_QPSK',
        f'Melbourne_{mode}_Thrpt',
        f'Melbourne_{mode}_Latency',
        f'Melbourne_{mode}_BER_QPSK'
    ]

    try:
        df = pd.read_csv(csv_file_path)
        print(f"Dataframe Columns ({mode}, {aqm}):", df.columns)
    except FileNotFoundError:
        print(f"Error: File not found: '{csv_file_path}'")
        return
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    if not all(col in df.columns for col in required_columns):
        missing = [col for col in required_columns if col not in df.columns]
        print(f"Error: Missing columns: {', '.join(missing)}")
        return

    script = [
        "#!/bin/bash",
        "set -x",
        "",
        "# Load environment settings",
        "if [ ! -f ../utils/settings.sh ]; then",
        "    echo \"Error: settings.sh not found\"",
        "    exit 1",
        "fi",
        "source ../utils/settings.sh",
        ""
    ]

    router_configs = []

    for i, row in df.iterrows():
        try:
            sydney_thrpt = pd.to_numeric(row[f'Sydney_{mode}_Thrpt'], errors='coerce')
            sydney_latency = pd.to_numeric(row[f'Sydney_{mode}_Latency'], errors='coerce')
            sydney_ber_qpsk = pd.to_numeric(row[f'Sydney_{mode}_BER_QPSK'], errors='coerce')

            melbourne_thrpt = pd.to_numeric(row[f'Melbourne_{mode}_Thrpt'], errors='coerce')
            melbourne_latency = pd.to_numeric(row[f'Melbourne_{mode}_Latency'], errors='coerce')
            melbourne_ber_qpsk = pd.to_numeric(row[f'Melbourne_{mode}_BER_QPSK'], errors='coerce')

            if any(pd.isna(val) for val in [
                sydney_thrpt, sydney_latency, sydney_ber_qpsk,
                melbourne_thrpt, melbourne_latency, melbourne_ber_qpsk
            ]):
                script.append(f"# Skipped datapoint {i + 1}: Invalid or missing data")
                continue

            thrpt = max(1, min(round(sydney_thrpt), round(melbourne_thrpt)))
            latency = round(sydney_latency + melbourne_latency)
            ber_qpsk = max(sydney_ber_qpsk, melbourne_ber_qpsk)
            loss_pct = ber_qpsk * 100  # keep decimals
            loss_pct = int(round(ber_qpsk * 100, 0))

            router_configs.append({
                'thrpt': thrpt,
                'latency': latency,
                'ber_qpsk': ber_qpsk,
                'loss_pct': loss_pct
            })

            script += [
                f"# --- Datapoint {i + 1} ---",
                f"echo \"Applying {thrpt} Mbps, {latency} ms, Loss: {loss_pct:.4f}% with {aqm}\"",
                "",
                "# Configure ens37 (delay + AQM)",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 root handle 1: netem delay {latency}ms loss {loss_pct}%\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens37 parent 1:1 handle 2: {aqm}\"",
                "",
                "# Configure ens38 (rate + AQM)",
                "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 root handle 1: tbf rate {thrpt}mbit burst 32kbit latency 100ms\"",
                f"ssh root@\"$router_ipaddr\" \"sudo tc qdisc add dev ens38 parent 1:1 handle 2: {aqm}\"",
                "",
                f"echo \"Completed setup for datapoint {i + 1}\"",
                "sleep 30",
                ""
            ]

        except Exception as e:
            script.append(f"# Skipped datapoint {i + 1}: Exception: {e}")
            continue

    pd.DataFrame(router_configs).to_csv(config_filename, index=False)

    script += [
        "# Cleanup",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens37 root 2>/dev/null || true\"",
        "ssh root@\"$router_ipaddr\" \"sudo tc qdisc del dev ens38 root 2>/dev/null || true\"",
        "echo \"All datapoints processed.\"",
        "exit 0"
    ]

    with open(script_filename, "w") as f:
        f.write("\n".join(script))

    print(f"✅ Bash script saved to: {script_filename}")
    print(f"✅ Router config saved to: {config_filename}")
    print("ℹ️  Make the script executable with: chmod +x", script_filename)


if __name__ == "__main__":
    aqm_list = ["fq_codel", "codel", "cake", "fq_pie", "pfifo","dualpi2"]
    for mode in ["Baseline", "Best"]:
        for aqm in aqm_list:
            generate_bash_script(mode, aqm, script_dir="scripts", config_dir="configs")


Dataframe Columns (Baseline, fq_codel): Index(['Time', 'Sydney_Baseline_SAT_ID', 'Sydney_Baseline_SNR',
       'Sydney_Baseline_RSSI', 'Sydney_Baseline_Thrpt',
       'Sydney_Baseline_BER_MQAM', 'Sydney_Baseline_BER_QPSK',
       'Sydney_Baseline_Latency', 'Melbourne_Baseline_SAT_ID',
       'Melbourne_Baseline_SNR', 'Melbourne_Baseline_RSSI',
       'Melbourne_Baseline_Thrpt', 'Melbourne_Baseline_BER_MQAM',
       'Melbourne_Baseline_BER_QPSK', 'Melbourne_Baseline_Latency'],
      dtype='object')
✅ Bash script saved to: scripts\apply_tc_rules_baseline_fq_codel.sh
✅ Router config saved to: configs\router_configs_baseline_fq_codel.csv
ℹ️  Make the script executable with: chmod +x scripts\apply_tc_rules_baseline_fq_codel.sh
Dataframe Columns (Baseline, codel): Index(['Time', 'Sydney_Baseline_SAT_ID', 'Sydney_Baseline_SNR',
       'Sydney_Baseline_RSSI', 'Sydney_Baseline_Thrpt',
       'Sydney_Baseline_BER_MQAM', 'Sydney_Baseline_BER_QPSK',
       'Sydney_Baseline_Latency', 'Melbourne_Bas